In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%cp -av /content/drive/MyDrive/rec/items.csv items.csv

'/content/drive/MyDrive/rec/items.csv' -> 'items.csv'


In [22]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import pickle

movies = pd.read_csv("items.csv")

In [23]:
title = movies.filter(['title','movie_id'],axis=1)
movies = movies.drop(['title','year','overview','vote_count','vote_average',],axis=1)
movies.head()

,movie_id,genre_unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western,original_language,runtime
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,en,81.0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,en,130.0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,en,98.0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,en,105.0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,en,124.0


In [24]:
title.head()

,title,movie_id
0,Toy Story (1995),1
1,GoldenEye (1995),2
2,Four Rooms (1995),3
3,Get Shorty (1995),4
4,Copycat (1995),5


In [25]:
movies.iloc[:,-2] = movies.iloc[:,-2].fillna('NaN')
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
movies.iloc[:,-2]=le.fit_transform(movies.iloc[:,-2])

In [26]:
movies = movies.fillna(movies.median())
movies.isnull().any()
movies.head()

,movie_id,genre_unknown,Action,Adventure,Animation,Children,Comedy,Crime,Documentary,Drama,Fantasy,FilmNoir,Horror,Musical,Mystery,Romance,SciFi,Thriller,War,Western,original_language,runtime
0,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,9,81.0
1,2,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,130.0
2,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,9,98.0
3,4,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,9,105.0
4,5,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0,9,124.0


In [27]:
model = NearestNeighbors(n_neighbors=20, algorithm="ball_tree").fit(movies)

with open('content_based.sav', 'wb') as pickle_out:
    pickle.dump(model, pickle_out)

In [28]:
distances, indices = model.kneighbors(movies,n_neighbors=11)

In [29]:
movies.to_pickle("content.pkl")
title.to_pickle("content_title.pkl")

In [30]:
input = "Toy Story"
input = title[title['title'].str.contains(input)]
input_id = input.iloc[0]['movie_id']
indices = indices[input_id-1]

In [31]:
recommend_frame = []
for val in indices[1:]:
    idx = title.iloc[val]['movie_id']
    recommend_frame.append({'Title':title.iloc[idx]['title']})
df = pd.DataFrame(recommend_frame,index=range(1,11))
df

,Title
1,Dead Man Walking (1995)
2,Get Shorty (1995)
3,Antonia's Line (1995)
4,"Postino, Il (1994)"
5,From Dusk Till Dawn (1996)
6,Copycat (1995)
7,Seven (Se7en) (1995)
8,Mighty Aphrodite (1995)
9,Braveheart (1995)
10,Angels and Insects (1995)
